# We want to build a chat style model, not completion style model. 
### In previous we have just completion style model where input -> model -> output
### Chat style model is different, because we have to discriminate between user message and chat message, there is also a system message that can specify how the model will behave

![title](pic/Screenshot_20240518_193150.png)

![title](pic/Screenshot_20240518_193621.png)

### ChatGPT does not store history, so we always have to include the previous history. 
### For that we need to use ChatPromptTemplate, not just PromptTemplate. Here is diagram how it works behind the scene

![title](pic/Screenshot_20240518_194410.png)


# Example with no memory involved - se the answer that are completely out of reality

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

load_dotenv()
chat = ChatOpenAI()

# a bit CONFUSING here - as this should hold the whole history
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[HumanMessagePromptTemplate.from_template("{content}")]
)

chain  = LLMChain(
    llm=chat,
    prompt=prompt,
)

res = None
while True:
    content = input(">> ")
    res = chain({"content": content})    
    print(res)
    
    if content == 'q':
        break

{'content': 'q', 'text': 'Hello! How can I assist you today?'}


# Memory = class that we can put inside a chain to keep track of history message. It stores data inside the chain. It is used twice 1]when we send question to the model, and 2] when we receive an answer
 
![title](pic/Screenshot_20240518_200338.png)

# So when we first call our chain this is what happens. The memory inside the model stores our message + some additional field ![title](pic/Screenshot_20240518_201145.png)

# When outputing the response it stores it also
![title](pic/Screenshot_20240518_201145.png) 

# There are many different kind of memory but mostly for completion models. We want chat based model so ConversationBufferMemory is used.

# So this is what is happening when we get response 
![title](pic/Screenshot_20240518_202302.png) 
# And this is when we continue replying
![title](pic/Screenshot_20240518_202525.png) 
# The last step is to send the history list to ChatGPT
![title](pic/Screenshot_20240518_202833.png) 
 

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

load_dotenv()
chat = ChatOpenAI()

memory = ConversationBufferMemory(
    memory_key="messages", # this is the key name that will be created inside the memory buffer
    return_messages=True  # dont just throw strings but wrap them into HumanMessage... for chat like models
)

# a bit CONFUSING here - as this should hold the whole history
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"), # this needs to match with the memory_key inside the ConversationBufferMemory
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain  = LLMChain(
    llm=chat,
    prompt=prompt,
    memory=memory
)

res = None
while True:
    content = input(">> ")
    res = chain({"content": content})    
    print(res)
    
    if content == 'q':
        break

{'content': 'what is 1+1?', 'messages': [HumanMessage(content='what is 1+1?'), AIMessage(content='1+1 equals 2.')], 'text': '1+1 equals 2.'}
{'content': 'and 3 more?', 'messages': [HumanMessage(content='what is 1+1?'), AIMessage(content='1+1 equals 2.'), HumanMessage(content='and 3 more?'), AIMessage(content='1+1+1 equals 3.')], 'text': '1+1+1 equals 3.'}
{'content': 'q', 'messages': [HumanMessage(content='what is 1+1?'), AIMessage(content='1+1 equals 2.'), HumanMessage(content='and 3 more?'), AIMessage(content='1+1+1 equals 3.'), HumanMessage(content='q'), AIMessage(content='Is there anything else I can assist you with?')], 'text': 'Is there anything else I can assist you with?'}
